This is our grid search document! We run grid search for ANN, Perceptron, and DNN. For ANN we test 972 hyperparameters (~ 1 hour to run), for perceptron we test 144 hyperparameters (~ 5-10 minutes to run), and for DNN we test 2916 hyperparameters (~ 6-7 hours to run)

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.errors_impl import InvalidArgumentError
import matplotlib.pyplot as plt

In [3]:
# Preprocessing + Training

# Load the dataset
df = pd.read_csv("../backend/Datasets/COALINDIA.csv")

# Feature selection and preprocessing
dropped_features = ['Date', 'Symbol', 'Series', 
                    'Trades', 'Turnover', 'Deliverable Volume', 
                    '%Deliverble', 'Last', 'VWAP', 'Prev Close']
df.drop(dropped_features, axis=1, inplace=True)

# Define features (X) and target (Y)
X = df.drop('Close', axis=1)
Y = df['Close']

# Scale the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X.values)
Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)

print(f"X_train - {X_train}")
print("*" * 50)
print(f"X_test - {X_test}")
print("*" * 50)

print(f"Y_train - {Y_train}")
print("*" * 50)

print(f"Y_test - {Y_test}")


X_train - [[0.53912913 0.69460631 0.5541304  1.        ]
 [0.69474787 0.72926939 0.71293327 0.06651257]
 [0.72108335 0.72747647 0.6717056  0.09779414]
 ...
 [0.40011971 0.40475123 0.4025042  0.02255257]
 [0.40026934 0.39489018 0.38616583 0.04220904]
 [0.36465659 0.36261766 0.36784242 0.02943177]]
**************************************************
X_test - [[0.36555439 0.37950097 0.37196519 0.01550283]
 [0.37827323 0.38936202 0.38097419 0.01553931]
 [0.37572946 0.37546691 0.37883646 0.00991563]
 ...
 [0.05132426 0.05079934 0.05481753 0.01289286]
 [0.05656142 0.05259226 0.05527561 0.01735301]
 [0.04952865 0.06618856 0.05329058 0.0570686 ]]
**************************************************
Y_train - [[0.69701067]
 [0.71894247]
 [0.66155926]
 ...
 [0.39762656]
 [0.37990086]
 [0.36472886]]
**************************************************
Y_test - [[3.80501728e-01]
 [3.78248460e-01]
 [3.72239748e-01]
 [3.77196936e-01]
 [3.80501728e-01]
 [3.78098242e-01]
 [3.78098242e-01]
 [3.71188223e-01]

In [3]:
def create_model_A(activation='relu', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, dropout=0.15):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(hidden, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [4]:
ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for dropout in [0.05, 0.1, 0.15]:
                        try:
                            mse, r2 = create_model_A(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, dropout=dropout)
                            ret.append([activation, epochs, batch_size, op_learning_rate, hidden, dropout, mse, r2])
                        except InvalidArgumentError:
                            print("Model error with params")

    

17/17 [==============================] - 0s 354us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.05
Mean Squared Error: 614.5569588877757
R^2: 0.6853248
----------------------------------------------------------------
17/17 [==============================] - 0s 336us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.1
Mean Squared Error: 630.9692290012133
R^2: 0.6769211
----------------------------------------------------------------
17/17 [==============================] - 0s 320us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.15
Mean Squared Error: 977.5246569718644
R^2: 0.49947238
----------------------------------------------------------------
17/17 [==============================] - 0s 339us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 64
Dropout: 0.05
Mean Squared Error: 780.134311756936
R^2: 0.60054326
----------------------------------------------------------------
17/17 [==============================] - 0s 332us/step


...

Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.05
Mean Squared Error: 8.05694802098225
R^2: 0.9958745
----------------------------------------------------------------
17/17 [==============================] - 0s 351us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.1
Mean Squared Error: 4.867682075246598
R^2: 0.9975076
----------------------------------------------------------------
17/17 [==============================] - 0s 363us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 64
Dropout: 0.15
Mean Squared Error: 10.375720999378366
R^2: 0.99468726
----------------------------------------------------------------
17/17 [==============================] - 0s 397us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 128
Dropout: 0.05
Mean Squared Error: 6.277810490538479
R^2: 0.9967855
----------------------------------------------------------------
17/17 [==============================] - 0s 346us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 128
Dropout: 0.1
Mean Squared Error: 4.745079257737809
R^2: 0.99757034
----------------------------------------------------------------
17/17 [==============================] - 0s 362us/step
Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Hidden: 128
Dropout: 0.15
Mean Squared Error: 2.196050037246792
R^2: 0.99887556
----------------------------------------------------------------


In [5]:
print("Best MSE for ANN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for ANN:", sorted(ret, key=lambda x:-x[-1])[0])

Best MSE for ANN: ['tanh', 150, 32, 0.001, 32, 0.05, 2.169791140612021, 0.99888897]
Best R^2 for ANN: ['tanh', 150, 32, 0.001, 32, 0.05, 2.169791140612021, 0.99888897]


In [6]:
def create_model_P(activation=None, epochs=100, batch_size=32, op_learning_rate=1e-5):
    model = Sequential()
    model.add(Dense(1, input_dim=X_train.shape[1], activation=activation))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    # Calculate MSE
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error (Perceptron): {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2


In [7]:
ret = []
for activation in [None, 'relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                try:
                    mse, r2 = create_model_P(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate)
                    ret.append([activation, epochs, batch_size, op_learning_rate, mse, r2])
                except InvalidArgumentError:
                    print("Model error with params")


17/17 [==============================] - 0s 302us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 7500.047842740202
R^2: -2.8402932
----------------------------------------------------------------
17/17 [==============================] - 0s 319us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 4040.1256514283964
R^2: -1.0686891
----------------------------------------------------------------
17/17 [==============================] - 0s 321us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 2717.9832912672973
R^2: -0.39170468
----------------------------------------------------------------
17/17 [==============================] - 0s 315us/step


Activation: None
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 40428.031501758865
R^2: -19.7006
----------------------------------------------------------------
17/17 [==============================] - 0s 320us/step


...

Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 153.90676985183433
R^2: 0.92119426
----------------------------------------------------------------
17/17 [==============================] - 0s 364us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 1857.1667322043188
R^2: 0.04906422
----------------------------------------------------------------
17/17 [==============================] - 0s 324us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 3048.2239231845965
R^2: -0.56079984
----------------------------------------------------------------
17/17 [==============================] - 0s 377us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 8756.81136260911
R^2: -3.4838004
----------------------------------------------------------------
17/17 [==============================] - 0s 359us/step


Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 2520.566410409547
R^2: -0.29062033
----------------------------------------------------------------
17/17 [==============================] - 0s 339us/step
Activation: tanh
Epoch: 150
Batch Size: 40
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 785.1469881433526
R^2: 0.5979766
----------------------------------------------------------------


In [8]:
print("Best MSE for Perceptron:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for Perceptron:", sorted(ret, key=lambda x:-x[-1])[0])

Best MSE for Perceptron: [None, 150, 24, 0.0001, 4.397121406273408, 0.9977485]
Best R^2 for Perceptron: [None, 150, 24, 0.0001, 4.397121406273408, 0.9977485]


In [4]:
def create_model_D(activation='tanh', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, layers=11, dropout=0.1):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    # Building DNN model with 11 hidden layers
    model = Sequential()

    # Input layer
    # Input layer will have same number of neurons as number of feature variables
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))

    # Hidden layers
    # Play around with number of neurons in each hidden layer.
    # Too many neurons leads to overcomplexity, not enough means too simple
    # Tanh activation function here is used b/c it is recommended to use
    # when there are more hidden layers.
    for _ in range(layers-1):
        model.add(Dense(hidden, activation=activation))
        model.add(Dropout(dropout)) # This helps with preventing overfitting

    # Output layer
    # Output layer will have 1 neuron b/c there's only 1 target variable
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nLayers: {layers}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [10]:
m, r = create_model_D()

17/17 [==============================] - 0s 574us/step
Activation: tanh
Epoch: 100
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 128
Layers: 11
Dropout: 0.1
Mean Squared Error: 40.36478406562411
R^2: 0.9793318
----------------------------------------------------------------


In [ ]:
# GRID SEARCH

ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for layers in [6, 9, 11]:
                        for dropout in [0.05, 0.1, 0.15]:
                            try:
                                mse, r2 = create_model_D(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, layers=layers, dropout=dropout)
                                ret.append([activation, epochs, batch_size, op_learning_rate, hidden, layers, dropout, mse, r2])
                            except InvalidArgumentError:
                                print("Model error with params")



17/17 [==============================] - 0s 412us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 6
Dropout: 0.05
Mean Squared Error: 2815.188583786327
R^2: -0.44147742
----------------------------------------------------------------
17/17 [==============================] - 0s 457us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 6
Dropout: 0.1
Mean Squared Error: 2175.2875816544415
R^2: -0.1138252
----------------------------------------------------------------
17/17 [==============================] - 0s 442us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 6
Dropout: 0.15
Mean Squared Error: 3677.793708039349
R^2: -0.883162
----------------------------------------------------------------
17/17 [==============================] - 0s 443us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 9
Dropout: 0.05
Mean Squared Error: 5331.1387113270375
R^2: -1.7297337
----------------------------------------------------------------
17/17 [==============================] - 0s 456us/step


...

Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 32
Layers: 11
Dropout: 0.1
Mean Squared Error: 737.8114196681706
R^2: 0.62221414
----------------------------------------------------------------
17/17 [==============================] - 0s 645us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 32
Layers: 11
Dropout: 0.15
Mean Squared Error: 3291.5653220935305
R^2: -0.6853993
----------------------------------------------------------------
17/17 [==============================] - 0s 437us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 64
Layers: 6
Dropout: 0.05
Mean Squared Error: 95.08504856845228
R^2: 0.9513131
----------------------------------------------------------------
17/17 [==============================] - 0s 389us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 64
Layers: 6
Dropout: 0.1
Mean Squared Error: 25.15586665923395
R^2: 0.9871193
----------------------------------------------------------------
17/17 [==============================] - 0s 392us/step


Activation: tanh
Epoch: 150
Batch Size: 32
Optimizer learning rate: 0.001
Hidden: 64
Layers: 6
Dropout: 0.15
Mean Squared Error: 151.78057217790393
R^2: 0.92228293
----------------------------------------------------------------


In [6]:
print("Best MSE for DNN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for DNN:", sorted(ret, key=lambda x:-x[-1])[0])

Best MSE for DNN: ['tanh', 100, 40, 0.0001, 128, 11, 0.05, 2.1822843321904517, 0.9988826]
Best R^2 for DNN: ['tanh', 100, 40, 0.0001, 128, 11, 0.05, 2.1822843321904517, 0.9988826]
